# Kalman as Bayesian Update with Gaussians 

## Two Prerequisites


### P1: Two Matrix  Identities

#### Id 1: Nameless
(Cheking the inverse annhilation, as $\text{LHS}*\text{RHS}^{-1}=\mathbb{1}$)
$$
(A+B)^{-1} = A^{-1}(A^{-1} + B^{-1})^{-1} B^{-1} 
$$
 

#### Id 2: [Woodbury Matrix Id](https://en.wikipedia.org/wiki/Woodbury_matrix_identity):
$$
(X^{-1} + Y^{-1})^{-1} = X - 
X  (X + Y)^{-1} X
$$

This follows form the full idenity, after the substuition of $(A,B,C,D)\hookrightarrow (X^{-1},\mathbb{1},Y^{-1},\mathbb{1})$:
$$
(A + BCD)^{-1} = A^{-1} - 
A^{-1} B (C^{-1} + D A^{-1} B)^{-1} D A^{-1} 
$$


### P2: Posterior = Likelihood $\times$ Prior, with the above identities

We have both:
1. $\mathcal{N}(\mu_0, \Sigma_0)$ 
2. $\mathcal{N}(\mu_1, \Sigma_1)$ 


The productor of the two: $\mathcal{N}(\mu_2, \Sigma_2)$, with
* $\Sigma_2^{-1} = \Sigma_0^{-1} + \Sigma_1^{-1}  $
* $\Sigma_2^{-1}\mu_2 = \Sigma_0^{-1} \mu_0 + \Sigma_1^{-1} \mu_1$

Using above Woodbury identy, it can be rewritten with $K = \Sigma_0 (\Sigma_0 +\Sigma_1)^{-1}$, as
- $\Sigma_2 = (\Sigma_0^{-1} + \Sigma_1^{-1})^{-1} 
\stackrel{\text{Woodbury}}{\Longrightarrow}
\Sigma_0  - K \Sigma_0$

- $\mu_2 = \Sigma_2 (\Sigma_0^{-1} \mu_0 + \Sigma_1^{-1} \mu_1) \stackrel{\text{Nameless}}{\Longrightarrow} 
\mu_0 + K(\mu_1-\mu_0)$




## Our System Specification 
(aka all the assumptions, for better or worse)

Model Specification:
With $X \sim \mathcal{N}(x,P)$ and $\xi \sim \mathcal{N}(0, Q)$, we have the evolution on top of which, leading to: 
$$
\begin{align}
X(t+1) &= A(t) X(t) +  \xi(t) \\
P(t+1) &= A(t) P(t) A(t)^T + Q(t) 
\end{align}
$$
Observation Specification: 
$$z(t+1)|_{X(t+1)} \sim \mathcal{N}(H(t) x(t+1), R(t+1)) $$



## Kalman Update, as Bayesian Update of Gaussians

From model specification, we have $(x_0,P_0)  \stackrel{\text{Model}}{\longrightarrow} (x_1,P_1)$:
* $x_1 \equiv \mathbb{E}(X_1) = F x_0$
* $P_1 = A P_0 A^T + Q$

From observed divation, we have to adjust $(x_1,P_1) \stackrel{\text{Obs}}{\longrightarrow}  (x_2,P_2)$
- Prior: $\mathcal{N}(H x_1, H P_1 H^T)$
- Likelihood: $\mathcal{N}(z,R)$

- Posterior: $\mathcal{N}(H x_2, H P_2 H^T)$
    * $K = H P_1 H^T (H P_1 H^T + R)^{-1}$
    * $H x_2 = H x_1 + K(z - H x_1 )$
    * $H P_2 H^T = H P_1 H^T - K H P_1 H^T$

    $\Rightarrow$ Refining $K = H K_G$, leading to,
        - $K_G = P_1 H^T (H P_1 H^T + R)^{-1}$
        - $x_2 = x_1 + K_G (z - H x_1 )$
        - $P_2 = P_1  - K_G H P_1 $
